In [1]:
import pandas as pd 
import packing_list_helper
from openpyxl import load_workbook
from openpyxl.styles import Border
from packing_list_helper import *

# Read the data file
rdc_df = pd.read_csv("data/rdc.csv")
item_list_df = pd.read_csv("data/item_list.csv")
po_details_df = pd.read_csv("data/po_details.csv")

In [2]:
rdc_df.rename(columns = {'出貨地代碼':'出貨'}, inplace = True)
rdc_df

,客戶別,出貨,出貨地,國別
0,TG004,TG0955,"MOUNT VERNON,TX",USA
1,TG004,TG0960,"STATESVILLE,NC",USA
2,TG004,TG0961,"MINERSVILLE,PA",USA
3,TG004,TG0962,"VALDOSTA,GA",USA
4,TG004,TG0964,"NORTH WILKESBORO,NC",USA
5,TG004,TG0965,"CHEYENNE,WY",USA
6,TG004,TG0966,"PERRIS,CA",USA
7,TG004,TG0990,"FINDLAY,OH",USA
8,TG004,TG0992,"NORTH VERNON,IN",USA
9,TG004,TG1419,"KISSIMMEE,FL",USA


In [3]:
# Modify the po detail csv
po_details_df.rename(columns = {'Item#':'ITEM#'}, inplace = True)
po_details_df.iloc[0]

採購單號                                                300063935
採購單日期                                               2022-9-21
客戶代號                                                    AS000
客戶PO單號                                                3007311
出貨                                                     TG0099
ITEM#                                                 4933155
品名          2"PVC 無拉仿木共押百葉窗(套管梯帶)-Trim@Home CF8_58"*48"_1E...
尺寸                                                    58"*48"
客戶產品顏色                                                   GRAY
出貨日期                                                2022-12-7
外箱 PC-IN                                                    2
數量                                                        174
(單位)                                                       EA
料號                                              9TG008FW24022
製作規範                                             TG008FW24B03
工廠代號                                                    CH001
Name: 0,

In [4]:
# handle the item_list
item_list_df["N.W.(KG)"] = item_list_df["G.W.(KG)"] - item_list_df["外箱"]
item_list_df["pvc/单"] = (item_list_df["N.W.(KG)"] / item_list_df["PC-IN"]) - item_list_df["配件重/单"] - item_list_df["内盒"]
item_list_df.shape

(38, 12)

In [9]:
# Data join to detail
new_po_detail_df = po_details_df.join(item_list_df.set_index('ITEM#'), on='ITEM#')
new_po_detail_df = new_po_detail_df.join(rdc_df.set_index('出貨'), on='出貨')
new_po_detail_df.columns

Index(['採購單號', '採購單日期', '客戶代號', '客戶PO單號', '出貨', 'ITEM#', '品名', '尺寸', '客戶產品顏色',
       '出貨日期', '外箱 PC-IN', '數量', '(單位)', '料號', '製作規範', '工廠代號',
       'Item Description', 'SIZE(inch)', 'COLOR', 'PC-IN', '配件重/单', '内盒', '外箱',
       'G.W.(KG)', 'CBM(M3)', 'N.W.(KG)', 'pvc/单', '客戶別', '出貨地', '國別'],
      dtype='object')

In [11]:
new_po_detail_df_group = new_po_detail_df.groupby(['客戶PO單號'])
wb = load_workbook('xlsx_template/packing_list_template.xlsx')
temple_sheet = wb.active
for name, group in new_po_detail_df_group:

    target = wb.copy_worksheet(temple_sheet)
    construct_customer_po_info(target,group.iloc[0])
    offset = 1
    row_count = 6
    
    for index, row in group.iterrows():
        construct_row_detail(target, row_count, row, offset)
        offset += row['數量']//row['外箱 PC-IN']
        row_count += 1
    construct_po_summery(target, row_count)
    target.title = str(name)
    
wb.remove_sheet(temple_sheet)
wb.save(f'results/details_po.xlsx')
wb.close

/var/folders/hj/yxkfnhm51b951nllng9gm0640000gn/T/ipykernel_86849/258998919.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in new_po_detail_df_group:
/var/folders/hj/yxkfnhm51b951nllng9gm0640000gn/T/ipykernel_86849/258998919.py:18: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  wb.remove_sheet(temple_sheet)


<bound method Workbook.close of <openpyxl.workbook.workbook.Workbook object at 0x11dc26350>>

In [ ]:
# detail = new_po_detail_df_group.get_group(226875745)
# print(detail.columns)
# for index, row in detail.iterrows():
#     print(row['數量'])
#     print(row['外箱 PC-IN'])
# # each_po_item_gp = detail.groupby(['ITEM#'])
# # each_po_item_gp.get_group(924684)


In [ ]:
# wb = load_workbook('xlsx_template/packing_list_template.xlsx')
# ws = wb.active
# ws.cell(row=4, column=3, value=300063935)
# ws.cell(row=4, column=4, value='=C4+123')
# wb.save("script_result.xlsx")